# 数据准备

In [1]:
import gurobipy
from gurobipy import GRB
from gurobipy import *
from gurobipy import LinExpr
import numpy as np
import re
# read the data

file_path = "./data_set/data1.txt"

node = [] # nodes
b = [] # cost to increase capacity
f = [] # cost to build new arcs
c = [] # cost to transport goods
arcs_A = [] # orignal arcs
arcs_B = [] # new arcs
arcs_AB = [] # all arcs
s = [] # source nodes
t = [] # target nodes
d = [] # demands
q = [] # capacity 
omega = [] # demands(transformation)
delta = [] # module_capacity

M = 10000000 # big M

with open(file_path, 'r') as file:
    lines = file.readlines()

for i in range(len(lines)):
    if lines[i] == "NODES (\n":
        j = i + 1
        while lines[j] != ")\n":
            if lines[j] == ")\n":
                break
            node.append(lines[j].split()[0])
            j += 1

n = len(node)
b = [[0 for i in range(n)] for j in range(n)]
q = [[0 for i in range(n)] for j in range(n)]
c = [[0 for i in range(n)] for j in range(n)]
delta = [[0 for i in range(n)] for j in range(n)]
f = [[0 for i in range(n)] for j in range(n)]

for i in range(len(lines)):
    if lines[i] == "LINKS (\n":
        j = i + 1
        while lines[j] != ")\n":
            link = lines[j].split()
            begin = link[2]
            end = link[3]
            begin_idx = node.index(begin) 
            end_idx = node.index(end)

            capacity = float(link[5])
            install_capacity_cost = float(link[6])
            route_cost = float(link[7])
            setup_cost = float(link[8])
            module_capacity = float(link[10])
            module_cost = float(link[11])

            arcs_AB.append((begin_idx, end_idx))
            arcs_AB.append((end_idx, begin_idx))
            if capacity == 0:
                arcs_B.append((begin_idx, end_idx))
                arcs_B.append((end_idx, begin_idx))
            else:
                arcs_A.append((begin_idx, end_idx))
                arcs_A.append((end_idx, begin_idx))
            q[begin_idx][end_idx] = capacity
            c[begin_idx][end_idx] = route_cost
            b[begin_idx][end_idx] = module_cost
            delta[begin_idx][end_idx] = module_capacity

            q[end_idx][begin_idx] = capacity
            c[end_idx][begin_idx] = route_cost
            b[end_idx][begin_idx] = module_cost
            delta[end_idx][begin_idx] = module_capacity
            
            j += 1
        i = j
    
    if lines[i] == "DEMANDS (\n":
        j = i + 1
        while lines[j] != ")\n":
            demand = lines[j].split()
            source = demand[2]
            target = demand[3]
            source_idx = node.index(source)
            target_idx = node.index(target)
            routing_unit = float(demand[5])
            demand_value = float(demand[6])

            s.append(source_idx)
            t.append(target_idx)
            d.append(demand_value)
            j += 1

D = len(d) # kinds of demands
omega = [ [0 for k in range(D)] for i in range(n)]
for k in range(D):
    for i in range(n):
        if i == s[k]:
            omega[i][k] = d[k]
        elif i == t[k]:
            omega[i][k] = -d[k]
        else:
            omega[i][k] = 0




# 定义子问题

In [ ]:
# 定义子问题
def sub_sol_with_mn (y, node, edge, omega, c, arcs_A, arcs_B, arcs_AB, d, delta, q):
    sub = gurobipy.Model("Sub Problem")
    sub.setParam('InfUnbdInfo', 1)
    n = len(node)
    D = len(d)
    # Add variables
    x = sub.addVars(n, n, D, vtype=GRB.CONTINUOUS, name="x")
    # Set objective
    obj = LinExpr()
    for i, j in arcs_AB:
        for k in range(D):
            obj += c[i][j] * x[i, j, k]
    sub.setObjective(obj, GRB.MINIMIZE)
    # Add constraints
    for (i, j) in arcs_AB:
        expr = LinExpr()
        for k in range(D):
            expr += x[i, j, k]
        
        if i == edge[0] and j == edge[1]:
            sub.addConstr(expr == 0, name="flow conservation-pi["+str(i)+","+str(j)+"]")
        elif (i, j) in arcs_B:
            sub.addConstr(expr <= y[i][j] * delta[i][j], name="flow conservation-pi["+str(i)+","+str(j)+"]")
        elif (i, j) in arcs_A:
            sub.addConstr(expr <= q[i][j] + y[i][j] * delta[i][j], name="flow conservation-pi["+str(i)+","+str(j)+"]")
    
    for i in range(n):
        for k in range(D):
            expr1 = LinExpr()
            expr2 = LinExpr()
            for j in range(n):
                expr1 += x[i, j, k]
                expr2 += x[j, i, k]
            sub.addConstr((expr1 - expr2) == omega[i][k], name="demand conservation-gamma["+str(i)+","+str(k)+"]")
        
    for i in range(n):
        for j in range(n):
            if (i, j) in arcs_AB:
                for k in range(D): 
                    sub.addConstr(x[i, j, k] >= 0, name="non-negativity["+str(i)+","+str(j)+","+str(k)+"]")
            else:
                for k in range(D): 
                    sub.addConstr(x[i, j, k] == 0, name="zero-flow["+str(i)+","+str(j)+","+str(k)+"]")
    
    sub.optimize()
    return sub

def dual_sub_sol_with_mn (y, node, edge, omega, c, arcs_A, arcs_B, arcs_AB, d, delta, q):
    sub = gurobipy.Model("Dual Sub Problem")
    sub.setParam('InfUnbdInfo', 1)
    n = len(node)
    D = len(d)
    # Add variables
    pi = sub.addVars(n, n, vtype=GRB.CONTINUOUS, name="pi")
    gamma = sub.addVars(n, D, vtype=GRB.CONTINUOUS, name="gamma")
    # Set objective
    obj = LinExpr()
    for i, j in arcs_AB:
        if i == edge[0] and j == edge[1]:
            obj += 0
        elif (i, j) in arcs_B:
            obj += delta[i][j] * y[i][j] * pi[i, j]
        elif (i, j) in arcs_A:
            obj += (q[i][j] + delta[i][j] * y[i][j]) * pi[i, j]
    for i in range(n):
        for k in range(D):
            obj += omega[i][k] * gamma[i, k]
    sub.setObjective(obj, GRB.MAXIMIZE)
    # Add constraints
    expr = LinExpr()
    for i, j in arcs_AB:
        expr += pi[i, j]

    for i, j in arcs_AB:
        for k in range(D):
            sub.addConstr(expr + gamma[i, k] - gamma[j, k] <= c[i][j], name="dual constraint")
    for i in range(n):
        for j in range(n):
            if (i, j) in arcs_AB:
                sub.addConstr(pi[i, j] <= 0, name="non-negativity")
            else:
                sub.addConstr(pi[i, j] == 0, name="non-negativity")
    
    sub.optimize()
    return sub

# test 求解带边失效原多商品网络流模型(子问题)

In [ ]:
y_vars = [[0 for i in range(n)] for j in range(n)]
with open ("./y_vars.txt", 'r') as file:
    lines = file.readlines()
    
    for i in range(len(lines)):
        line = lines[i]
        indices = re.findall(r'\d+', line)
        i = int(indices[0])
        j = int(indices[1])
        val = int(indices[2])
        y_vars[i][j] = val


sub = sub_sol_with_mn(y_vars, node, (0, 0), omega, c, arcs_A, arcs_B, arcs_AB, d, delta, q)

if sub.Status == GRB.INFEASIBLE:
    # add feasible cuts
    pi = [[0 for i in range(n)] for j in range(n)]
    gamma = [[0 for k in range(D)] for i in range(n)]
    # 获得极射线
    for con in sub.getConstrs():
        name = con.ConstrName
        if name == "non-negativity":
            break
        indices = re.findall(r'\d+', name)
        i = int(indices[0])
        j = int(indices[1])
        if con.ConstrName.startswith("flow conservation-pi"):
            pi[i][j] = con.farkasdual
        if con.ConstrName.startswith("demand conservation-gamma"):
            gamma[i][j] = con.farkasdual

sub = dual_sub_sol_with_mn(y_vars, node, (0, 0), omega, c, arcs_A, arcs_B, arcs_AB, d, delta, q)
pi_dual = [[0 for i in range(n)] for j in range(n)]
gamma_dual = [[0 for k in range(D)] for i in range(n)]
for v in sub.getVars():
    name = v.VarName
    indices = re.findall(r'\d+', name)
    i = int(indices[0])
    j = int(indices[1])

    if name.startswith("pi"):
        pi_dual[i][j] = v.x
    if name.startswith("gamma"):
        gamma_dual[i][j] = v.x

# 更新上下界

In [ ]:
# 主问题和callback定义
master = gurobipy.Model("Master Probelm")
master.setParam('LazyConstraints', 1)
# Add variables
y = master.addVars(n, n, lb = 0,vtype=GRB.INTEGER, name="y")
z = master.addVars(n, n, vtype=GRB.BINARY, name="z")
mu = master.addVar(vtype=GRB.CONTINUOUS, name="mu")

# updata the variables
master.update()

# Set objective
obj = LinExpr()
obj += mu
for i, j in arcs_AB:
    obj += b[i][j] * y[i, j]
for i, j in arcs_B:
    obj += f[i][j] * z[i, j]    

master.setObjective(obj, GRB.MINIMIZE)

# Add constraints

for i, j in arcs_B:
    master.addConstr(y[i, j] <= M * z[i, j], name="build_increase conflict")
for i in range(n):
    for j in range(n):
        if (i, j) in arcs_B:
            master.addConstr(y[i, j] <= M * z[i, j], name="build_increase conflict")
            
        elif (i, j) in arcs_A:
            master.addConstr(y[i, j] >= 0, name="non-negativity")
            master.addConstr(z[i, j] == 0)
        else:
            master.addConstr(y[i, j] == 0, name="non-negativity")
            master.addConstr(z[i, j] == 0)

In [ ]:
y_vars = [[0 for i in range(n)] for j in range(n)]
pi = [[0 for i in range(n)] for j in range(n)]
gamma = [[0 for k in range(D)] for i in range(n)]
# with open ("./y_vars.txt", 'r') as file:
#     lines = file.readlines()
    
#     for i in range(len(lines)):
#         line = lines[i]
#         indices = re.findall(r'\d+', line)
#         i = int(indices[0])
#         j = int(indices[1])
#         val = int(indices[2])
#         y_vars[i][j] = val
mu_vars = 0
UB = 100000000000
LB = 0
eps = 1e-6
while UB - LB > eps:
    print("UB: ", UB)
    print("LB: ", LB)
    for edge in arcs_AB:
        sub = sub_sol_with_mn(y_vars, node, edge, omega, c, arcs_A, arcs_B, arcs_AB, d, delta, q)
        
        if sub.Status == GRB.INFEASIBLE:
            # add feasible cuts
            # 获得极射线
            for con in sub.getConstrs():
                name = con.ConstrName
                if name == "non-negativity":
                    break
                indices = re.findall(r'\d+', name)
                i = int(indices[0])
                j = int(indices[1])
                if con.ConstrName.startswith("flow conservation-pi"):
                    pi[i][j] = con.farkasdual
                if con.ConstrName.startswith("demand conservation-gamma"):
                    gamma[i][j] = con.farkasdual
                
            expr = LinExpr()
            val = 0
            for i, j in arcs_AB:
                if i == edge[0] and j == edge[1]:
                    expr += 0
                elif (i, j) in arcs_B:
                    expr += y[i,j] * delta[i][j] * pi[i][j]
                    val += y_vars[i][j] * delta[i][j] * pi[i][j]
                elif (i, j) in arcs_A:
                    expr += (q[i][j] + y[i,j] * delta[i][j]) * pi[i][j]
                    val += (q[i][j] + y_vars[i][j] * delta[i][j]) * pi[i][j]
            for i in range(n):
                for k in range(D):
                    expr += omega[i][k] * gamma[i][k]
            master.addConstr(0 >= expr, name="Feasible cut")
            break
        elif sub.objVal > mu_vars:
            # add optimal cuts
            pi = [[0 for i in range(n)] for j in range(n)]
            gamma = [[0 for k in range(D)] for i in range(n)]
            # 获得对偶变量
            for con in sub.getConstrs():
                name = con.ConstrName
                if name == "non-negativity":
                    break
                indices = re.findall(r'\d+', name)
                i = int(indices[0])
                j = int(indices[1])
                if con.ConstrName.startswith("flow conservation-pi"):
                    pi[i][j] = con.Pi
                if con.ConstrName.startswith("demand conservation-gamma"):
                    gamma[i][j] = con.Pi

            expr = LinExpr()
            val = 0
            for i, j in arcs_AB:
                if i == edge[0] and j == edge[1]:
                    expr += 0
                    val += 0
                elif (i, j) in arcs_B:
                    expr += y[i, j] * delta[i][j] * pi[i][j]
                    val += y_vars[i][j] * delta[i][j] * pi[i][j]
                elif (i, j) in arcs_A:
                    expr += (q[i][j] + y[i, j] * delta[i][j]) * pi[i][j]
                    val += (q[i][j] + y_vars[i][j] * delta[i][j]) * pi[i][j]
            
            for i in range(n):
                for k in range(D):
                    expr += omega[i][k] * gamma[i][k]
                    val += omega[i][k] * gamma[i][k]
            master.addConstr(mu >= expr, name="Optimal cut")
            
            UB = sub.objVal + val

            break
            
    master.optimize()
    for v in master.getVars():
        if v.x!=0:
            print('%s %g' % (v.varName, v.x))
        if v.varName == "mu":
            mu_vars = mu.x
        if v.VarName.startswith("y"):
            indices = re.findall(r'\d+', v.varName)
            i = int(indices[0])
            j = int(indices[1])
            y_vars[i][j] = v.x
    LB = master.objVal

# 带callback的MP求解

In [ ]:
# 主问题和callback定义
master = gurobipy.Model("Master Probelm")
# Add variables
y = master.addVars(n, n, lb = 0,vtype=GRB.INTEGER, name="y")
z = master.addVars(n, n, vtype=GRB.BINARY, name="z")
mu = master.addVar(vtype=GRB.CONTINUOUS, name="mu")

# updata the variables
master.update()

# Set objective
obj = LinExpr()
obj += mu
for i, j in arcs_AB:
    obj += b[i][j] * y[i, j]
for i, j in arcs_B:
    obj += f[i][j] * z[i, j]    

master.setObjective(obj, GRB.MINIMIZE)

# Add constraints
# expr = LinExpr()
# for i, j in arcs_AB:
#     if i == edge[0] and j == edge[1]:
#         expr += 0
#     elif (i, j) in arcs_B:
#         expr += y[i,j] * delta[i][j] * pi[i][j]
#     elif (i, j) in arcs_A:
#         expr += (q[i][j] + y[i,j] * delta[i][j]) * pi[i][j]

# for i in range(n):
#     for k in range(D):
#         expr += omega[i][k] * gamma[i][k]

# master.addConstr(mu >= expr, name="Benders Cutting plane")

for i in range(n):
    for j in range(n):
        if (i, j) in arcs_B:
            master.addConstr(y[i, j] <= M * z[i, j], name="build_increase conflict")
            
        elif (i, j) in arcs_A:
            master.addConstr(y[i, j] >= 0, name="non-negativity")
            master.addConstr(z[i, j] == 0, name="zero")
        else:
            master.addConstr(y[i, j] == 0, name="zero")
            master.addConstr(z[i, j] == 0, name="zero")


def mycallback(master, where):
    if where == GRB.Callback.MIPSOL:
        sol = master.cbGetSolution(master._vars)
        length = len(sol)
        half = (length - 1) // 2
        mu_vars = sol[length - 1]
        n = int(np.sqrt(half))
        y_vars = [[0 for i in range(n)] for j in range(n)]

        for i in range(n):
            for j in range(n):
                y_vars[i][j] = sol[n*i + j]
        print(y_vars)
        for edge in arcs_A:
            sub = sub_sol_with_mn(y_vars, node, edge, omega, c, arcs_A, arcs_B, arcs_AB, d, delta, q)
            if sub.Status == GRB.INFEASIBLE:
                # add feasible cuts
                pi = [[0 for i in range(n)] for j in range(n)]
                gamma = [[0 for k in range(D)] for i in range(n)]
                # 获得极射线
                for con in sub.getConstrs():
                    name = con.ConstrName
                    if name == "non-negativity":
                        break
                    indices = re.findall(r'\d+', name)
                    i = int(indices[0])
                    j = int(indices[1])
                    if con.ConstrName.startswith("flow conservation-pi"):
                        pi[i][j] = con.farkasdual
                    if con.ConstrName.startswith("demand conservation-gamma"):
                        gamma[i][j] = con.farkasdual

                val = 0
                for i, j in arcs_AB:
                    if i == edge[0] and j == edge[1]:
                        val += 0
                    elif (i, j) in arcs_B:
                        val += y_vars[i][j] * delta[i][j] * pi[i][j]
                    elif (i, j) in arcs_A:
                        val += (q[i][j] + y_vars[i][j] * delta[i][j]) * pi[i][j]
                
                for i in range(n):
                    for k in range(D):
                        val += omega[i][k] * gamma[i][k]
                print(val)
                
                expr = LinExpr()
                for i, j in arcs_AB:
                    if i == edge[0] and j == edge[1]:
                        expr += 0
                    elif (i, j) in arcs_B:
                        expr += master._vars[n*i + j] * delta[i][j] * pi[i][j]
                    elif (i, j) in arcs_A:
                        expr += (q[i][j] + master._vars[n*i + j] * delta[i][j]) * pi[i][j]
                
                for i in range(n):
                    for k in range(D):
                        expr += omega[i][k] * gamma[i][k]
    
                master.cbLazy(expr >= 0)
                master.cbLazy(master._vars[0] >= 1)
                break
               
            elif sub.objVal > mu_vars:
                # add optimal cuts
                pi = [[0 for i in range(n)] for j in range(n)]
                gamma = [[0 for k in range(D)] for i in range(n)]
                # 获得对偶变量
                for con in sub.getConstrs():
                    name = con.ConstrName
                    if name == "non-negativity":
                        break
                    indices = re.findall(r'\d+', name)
                    i = int(indices[0])
                    j = int(indices[1])
                    if con.ConstrName.startswith("flow conservation-pi"):
                        pi[i][j] = con.Pi
                    if con.ConstrName.startswith("demand conservation-gamma"):
                        gamma[i][j] = con.Pi

                expr = LinExpr()
                for i, j in arcs_AB:
                    if i == edge[0] and j == edge[1]:
                        expr += 0
                    elif (i, j) in arcs_B:
                        expr += master._vars[15*i + j] * delta[i][j] * pi[i][j]
                    elif (i, j) in arcs_A:
                        expr += (q[i][j] + master._vars[15*i + j] * delta[i][j]) * pi[i][j]
                
                for i in range(n):
                    for k in range(D):
                        expr += omega[i][k] * gamma[i][k]

                master.cbLazy(master._vars[length - 1] >= expr)
                break
                               
master._vars = master.getVars()
master.Params.lazyConstraints = 1
# master.optimize()
master.optimize(mycallback)

for v in master.getVars():
    if v.x != 0:
        print('%s %g' % (v.varName, v.x))

